Created on Friday 07 January 2021

Group 5 - Classification
Extraction features syntaxe

@authors : L.D. and F.B

In [ ]:
import pandas as pd 
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import f1_score
from string import punctuation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_analyse = pd.read_json("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/df_final_clean.json")
df_analyse.drop('art_content', axis=1, inplace=True)

df_train = pd.read_csv("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/Data_With_Features_Syntax.csv")

df_train = pd.merge(left=df_analyse, right=df_train, left_on='art_id', right_on='art_id')

df_lexique: pd.DataFrame = pd.read_csv("/content/drive/MyDrive/G5 Inter-Promo 2021/Ressources/Lexique_Innovation.txt", sep="  ", header=None)
df_lexique.columns: list = ['key_word']

list_key_word = df_lexique.key_word.values.tolist()


In [ ]:
# Counts the number of words
def nb_word(text: str) -> int:
    """Documentation
      Parameters:
        text: text of the article

      Out (if exists):
        nb_word: number of word in  the text
    """
    nb_words: list = []
    nb: int = 0
    # removes special characters
    for p in punctuation:
      text= text.replace(p, ' ')

    # counts the number of words present in the text
    return len(text.split())


In [ ]:
'''
def clean_keyword(list_key_word : list):
  list_key_word_net : list = []
  for k in list_key_word :
    k=k.lower()
    k=k.replace(' de ',' ')
    k=k.replace(" d' ",' ')
    k=k.replace("."," ")
    k=k.replace("-"," ")
    k=k.replace(" du "," ")
    list_key_word_net.append(k)
  list_key_word_net = pd.unique(list_key_word_net).tolist()
  return list_key_word

#list_key_word = clean_keyword(list_key_word)
'''

In [ ]:
# Count the number of time where the words in the list appear

def count_key_words(text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        t : number of key word in  the text
    """
    t : int = 0

    if text != None:
        text = text.lower()
        text = text.split()
        for j in range(len(text)):
            if (text[j] in list_key_word):   #list_key_word: List of word that we will check in the sentences
                t += 1
            else : # For 2-word keywords
              try : # In case j+1 does not exist
                if text[j]+' '+text[j+1] in list_key_word :
                  t += 1
                else : #For 3-word keywords
                  try : # In case j+2 does not exist
                    if text[j]+' '+text[j+1]+' '+text[j+2] in list_key_word :
                      t+=1
                  except :
                    pass
              except :
                pass
    return t

SyntaxError: ignored

In [ ]:
# Count the number of sentence
def phrases( text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        n:  The number of sentence in a article
    """

    n : int = 0
   
    if not isinstance(text, str):
        text: str = str(text)

    if (text != None):
        text = text.replace("..", ".")
        text = text.replace("...", ".")
        text = text.replace("!", ".")
        text = text.replace("!!", ".")
        text = text.replace("!!!", ".")
        text = text.replace("?", ".")
        text = text.replace("??", ".")
        text = text.replace("???", ".")
        text = text.replace("?!", ".")
        text = text.replace("!?", ".")
        n = len(sent_tokenize(text))

    return n

In [ ]:
def innovation(data_nb1, data_nb2, data_ratio1, data_ratio2, data_ratio3):
  res = []
  seuil = 0.05
  for i in tqdm(range(len(data_nb1))):
    valeur = data_nb1[i]*data_ratio1[i] + 0.5*(data_nb2[i]*data_ratio2[i]) + 0.1*data_ratio3[i]
    if valeur > seuil :
      res.append(1)
    elif ((valeur < seuil) & (data_nb1[i] == 0) & (data_nb2[i] == 0)):
      res.append(0)
    elif ((valeur < seuil) & (data_ratio1[i] < 0.0005) & (data_nb2[i] < 0.001)):
      res.append(0)
    else :
      res.append('?')
  return res

In [ ]:
def score_innovation(data: pd.DataFrame, weight_content: float , weight_title:float)-> pd.DataFrame:
  ''' This function calculates the score for each article
    Input :
      weight_content : weight of the content
      weight_title : weight of the title
    Output:
      data : dataframe with the label_innovation score column 
  '''
  # centred and reduced
  data_scale : pd.DataFrame= data[['Nb_key_words','Nb_key_words_title']]
  data_scale = scaler.fit_transform(data_scale)

  # calculate the score 
  data['label_innovation'] = data_scale[:,0]*weight_content + data_scale[:,1]*weight_title # innovation score 

  # Extend over the interval 0 and 1
  i=max(data['label_innovation'].values)
  j=min(data['label_innovation'].values)
  data['label_innovation']=(data['label_innovation'] - j)/(i - j)

  return data


def classification_innovation( score_threshold : tuple) -> int:
  ''' This function classifies each article, 1 if it is innovative 0 otherwise
     Input :
        score_threshold : tuple containing the score and the threshold 
  '''
  score, threshold = score_threshold[0], score_threshold[1]
  if score <= threshold:
    return 0
  else:
    return 1


def classification_innovation_2(data: pd.DataFrame, weight_content: float = 0.7, weight_title:float = 0.3, threshold:float = 0.6)-> pd.DataFrame :
  ''' This function calculates the innovation score and then classifies each article. 
    Input:
      threshold : threshold , default = 0.6 : optimum thresholding
      weight_content : weight of the content, default = 0.7 : the optimum weight
      weight_title : weight of the title, default = 0.3 : the optimum weight
    Output:
      data : dataframe with the label_innovation score column 
  '''
  data = score_innovation(data, weight_content, weight_title)
  data['label_innovation']=list(zip(data['label_innovation'].values, np.repeat(threshold,len(data))))
  data['label_innovation']=data['label_innovation'].apply(classification_innovation)
  return data

In [ ]:
def create_features(data : pd.DataFrame, text : str, title : str):
  #
  data["Nb_key_words"] : np.DataFrame = data[text].apply(count_key_words)
  data["Nb_key_words_title"] : np.DataFrame = data[title].apply( count_key_words)
  data["Nb_words"]: np.DataFrame = data[text].apply(nb_word)
  data["Nb_words_title"]: np.DataFrame = data[title].apply(nb_word)
  data["Nb_sentences"]: np.DataFrame = data['art_content'].apply(phrases)
  data["average_word_sentence"]: np.DataFrame = data["Nb_words"] / data["Nb_sentences"]
  data['ratio_key_words']: np.DataFrame  = data['Nb_key_words']/data['Nb_words']
  data['ratio_key_words']: np.DataFrame  = data['ratio_key_words'].fillna(0)
  data['ratio_key_sentences']: np.DataFrame  = data['Nb_key_words']/data['Nb_sentences']
  data['ratio_key_sentences']: np.DataFrame  = data['ratio_key_sentences'].fillna(0)
  data['ratio_key_word_title']: np.DataFrame  = data['Nb_key_words_title'] / data['Nb_words_title']
  data['ratio_key_word_title']: np.DataFrame  = data['ratio_key_word_title'].fillna(0)

  #
  data['innovation'] = innovation(data['Nb_key_words'],data['ratio_key_words'],data['Nb_key_words_title'],data['ratio_key_word_title'],data['ratio_key_sentences'])
  #

  data = classification_innovation_2 (data = data)
  return data

In [ ]:
%%time
df_train = create_features(df_train, 'art_content_prepd', 'art_title')

NameError: ignored

In [ ]:
var_utile = df_train[['art_id','Nb_key_words','Nb_key_words_title','ratio_key_words','ratio_key_word_title']]

In [ ]:
def create_label(features : list, df_train : pd.DataFrame, data_type : str):
  df_train = df_train.sample(frac=1, random_state=15).reset_index(drop=True)
  df_train['index'] = df_train.index
  
  var_utile = df_train[features]
  var_utile = pd.concat([var_utile, df_train['innovation']], axis = 1)
  
  tout = var_utile[var_utile['innovation'] != '?']
  unlabeled = var_utile[var_utile['innovation'] == '?']
  
  X_train = tout.drop('innovation', axis=1)
  y_train = tout.innovation
  y_train = pd.to_numeric(y_train)

  X_unlabeled = unlabeled.drop('innovation', axis=1)

  if (data_type == 'key'):
    model1 = LogisticRegression(C=1e5)
    model2 = KNeighborsClassifier()
    model3 = make_pipeline(StandardScaler(),
                     SGDClassifier(max_iter=1000, tol=1e-3,loss = 'log'))
  elif (data_type == 'BOW'):
    model1 = svm.SVC(C = 4,kernel='linear', probability= True)
    model2 = LogisticRegression(C=1e5)
    model3 = make_pipeline(StandardScaler(),
                     SGDClassifier(max_iter=1000, tol=1e-3,loss = 'log'))
  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model1.fit(X_train, y_train)
    y_hat_train = model1.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data

      pred_probs : np.ndarray = model1.predict_proba(X_unlabeled)
      preds : np.ndarray = model1.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 99% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.99],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.99]],
                          axis=0)
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []

  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model2.fit(X_train, y_train)
    y_hat_train = model2.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data

      pred_probs : np.ndarray = model2.predict_proba(X_unlabeled)
      preds : np.ndarray = model2.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 99% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.99],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.99]],
                          axis=0)
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []

  # Initiate iteration counter
  iterations = 0

  # Containers to hold f1_scores and # of pseudo-labels
  train_f1s = []
  pseudo_labels = []

  # Assign value to initiate while loop
  high_prob = [1] 

  # Loop will run until there are no more high-probability pseudo-labels
  while len(high_prob) > 0:
        
    # Fit classifier and make train/test predictions
    model3.fit(X_train, y_train)
    y_hat_train = model3.predict(X_train)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(y_train, y_hat_train)
    train_f1s.append(train_f1)
   
    if (len(X_unlabeled) > 0):
      # Generate predictions and probabilities for unlabeled data

      pred_probs : np.ndarray = model3.predict_proba(X_unlabeled)
      preds : np.ndarray = model3.predict(X_unlabeled)
      prob_0 : list = pred_probs[:,0]
      prob_1 : list = pred_probs[:,1]

      # Store predictions and probabilities in dataframe
      df_pred_prob : pd.DataFrame = pd.DataFrame([])
      df_pred_prob['preds'] = preds
      df_pred_prob['prob_0'] = prob_0
      df_pred_prob['prob_1'] = prob_1
      df_pred_prob.index = X_unlabeled.index
    
      # Separate predictions with > 60% probability
      high_prob : pd.DataFrame = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.60],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.60]],
                          axis=0)
    
      pseudo_labels.append(len(high_prob))

      # Add pseudo-labeled data to training data
      X_train = pd.concat([X_train, X_unlabeled.loc[high_prob.index]], axis=0)
      y_train = pd.concat([y_train, high_prob.preds])      
    
      # Drop pseudo-labeled instances from unlabeled data
      X_unlabeled = X_unlabeled.drop(index=high_prob.index)
    
      # Update iteration counter
      iterations += 1
    else :
      high_prob = []

  X_train['innovation'] = y_train
  X_train['index'] = X_train.index
  df_final = pd.merge(left=df_train, right=X_train, left_on='index', right_on='index')
  df_final = df_final[['art_id','innovation_y']]
  df_final = df_final.rename(columns = {'innovation_y': 'innovation'})
  return df_final

In [ ]:
df_train_supervised = create_label(['Nb_key_words','Nb_key_words_title','ratio_key_words','ratio_key_word_title'], df_train, 'key')

KeyError: ignored

In [ ]:
df_train.drop('innovation', axis=1, inplace=True)
df_train = df_train.merge(df_train_supervised)

df_train.to_csv('/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Output/Data_With_Features_Syntax_and_Label.csv', index=False)